# My Argo Climatology

Originally I was using RG climatology, however that has many more data gaps than the dataset I've created and am working with. So, I'm creating a new climatology for the vertical coordinate interpolation problem. I'm calling this AMF_clim

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.path import Path
import matplotlib.colors as colors
import pandas as pd
import numpy as np
from importlib import reload
import cartopy.crs as ccrs
import cmocean.cm as cmo
import gsw
import scipy.ndimage as filter
import scipy.interpolate as interpolate
from flox.xarray import xarray_reduce
from xgcm.autogenerate import generate_grid_ds
from xgcm import Grid
from tqdm.notebook import tqdm
from scipy.ndimage import uniform_filter1d

In [3]:
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.titlesize']  = 18
plt.rcParams['axes.labelsize']  = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14

In [4]:
ds_p = xr.open_dataset('/swot/SUM05/amf2288/res_2.5/ds_p.nc').rename({'PRES_INTERPOLATED':'PRESSURE','N_PROF':'PROFILE','LATITUDE':'LAT','LONGITUDE':'LON'})   
ds_p


<xarray.Dataset> Size: 95GB
Dimensions:          (PROFILE: 1488063, PRESSURE: 1001)
Coordinates:
  * PRESSURE         (PRESSURE) int64 8kB 0 2 4 6 8 ... 1992 1994 1996 1998 2000
    TIME             (PROFILE) datetime64[ns] 12MB ...
    LAT              (PROFILE) float64 12MB ...
    LON              (PROFILE) float64 12MB ...
  * PROFILE          (PROFILE) int64 12MB 2428701 1114461 ... 1502504 411205
Data variables:
    CT               (PROFILE, PRESSURE) float64 12GB ...
    CYCLE_NUMBER     (PROFILE, PRESSURE) float64 12GB ...
    PLATFORM_NUMBER  (PROFILE, PRESSURE) float64 12GB ...
    PRES             (PROFILE, PRESSURE) float32 6GB ...
    PSAL             (PROFILE, PRESSURE) float32 6GB ...
    SA               (PROFILE, PRESSURE) float64 12GB ...
    SIG0             (PROFILE, PRESSURE) float64 12GB ...
    TEMP             (PROFILE, PRESSURE) float32 6GB ...
    sample_rate      (PROFILE, PRESSURE) float32 6GB ...
    SPICE            (PROFILE, PRESSURE) float64 12GB ...
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://data-argo.ifremer.fr
    Fetched_by:           amf2288
    Fetched_date:         2025/03/11
    Fetched_constraints:  [x=0.00/5.00; y=40.00/45.00; z=0.0/2002.0]
    Fetched_uri:          
    Valid uris:           ['https://data-argo.ifremer.fr/dac/coriolis/6903068...
    All uris:             ['https://data-argo.ifremer.fr/dac/coriolis/6903068...
    Processing_history:   [PRES,TEMP,PSAL] real-time and adjusted/delayed var...

In [5]:
def get_ds_gridded(ds, lon_bins, lat_bins, z_coord):
    pbar = tqdm(total=4, desc="Gridding dataset", unit="step")
    
    # Step 1: Reduce using flox
    ds_binned = xarray_reduce(
        ds,
        'LON',
        'LAT',
        func='mean',
        expected_groups=(
            pd.IntervalIndex.from_breaks(lon_bins),
            pd.IntervalIndex.from_breaks(lat_bins)
        ),
        fill_value=np.nan,
        skipna=True
    )
    pbar.update(1)

    # Step 2: Rename dimensions and variables
    ds_binned = ds_binned.rename_dims({
        'LON_bins': 'LON',
        'LAT_bins': 'LAT',}).rename_vars({
        #'rho_grid': 'density'
        'LON_bins': 'LON',
        'LAT_bins': 'LAT',
        #'rho_grid': 'density'
    })
    pbar.update(1)

    # Step 3: Replace Interval coordinates with midpoints
    def interval_midpoints(intervals):
        return np.array([interval.mid for interval in intervals])

    ds_binned = ds_binned.assign_coords({
        'LON': ('LON', interval_midpoints(ds_binned['LON'].values)),
        'LAT': ('LAT', interval_midpoints(ds_binned['LAT'].values))
    })
    pbar.update(1)

    # Step 4: Generate grid and xgcm object
    ds_gridded = generate_grid_ds(ds_binned, {'X': 'LON', 'Y': 'LAT'})
    grid = Grid(ds_gridded, coords={
        'X': {'center': 'LON', 'left': 'LON_left'},
        'Y': {'center': 'LAT', 'left': 'LAT_left'},
        'Z': {'center': z_coord}
    }, periodic=['X'])
    pbar.update(1)
    
    pbar.close()

    return ds_gridded, grid

In [6]:
lon_bins = np.arange(-180,181,3)
lat_bins = np.arange(-90,91,3)

In [7]:
AMF_clim, grid = get_ds_gridded(ds_p, lon_bins, lat_bins, z_coord='PRESSURE')

Gridding dataset:   0%|          | 0/4 [00:00<?, ?step/s]

In [10]:
AMF_clim.to_netcdf('/swot/SUM05/amf2288/AMF_clim_current.nc')

In [11]:
AMF_clim

<xarray.Dataset> Size: 461MB
Dimensions:          (PRESSURE: 1001, LON: 120, LAT: 60, LON_left: 120,
                      LAT_left: 60)
Coordinates:
  * PRESSURE         (PRESSURE) int64 8kB 0 2 4 6 8 ... 1992 1994 1996 1998 2000
  * LON              (LON) float64 960B -178.5 -175.5 -172.5 ... 175.5 178.5
  * LAT              (LAT) float64 480B -88.5 -85.5 -82.5 ... 82.5 85.5 88.5
  * LON_left         (LON_left) float64 960B -180.0 -177.0 ... 174.0 177.0
  * LAT_left         (LAT_left) float64 480B -90.0 -87.0 -84.0 ... 84.0 87.0
Data variables:
    CT               (PRESSURE, LON, LAT) float64 58MB nan nan nan ... nan nan
    CYCLE_NUMBER     (PRESSURE, LON, LAT) float64 58MB nan nan nan ... nan nan
    PLATFORM_NUMBER  (PRESSURE, LON, LAT) float64 58MB nan nan nan ... nan nan
    PRES             (PRESSURE, LON, LAT) float32 29MB nan nan nan ... nan nan
    PSAL             (PRESSURE, LON, LAT) float32 29MB nan nan nan ... nan nan
    SA               (PRESSURE, LON, LAT) float64 58MB nan nan nan ... nan nan
    SIG0             (PRESSURE, LON, LAT) float64 58MB nan nan nan ... nan nan
    TEMP             (PRESSURE, LON, LAT) float32 29MB nan nan nan ... nan nan
    sample_rate      (PRESSURE, LON, LAT) float32 29MB nan nan nan ... nan nan
    SPICE            (PRESSURE, LON, LAT) float64 58MB nan nan nan ... nan nan
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://data-argo.ifremer.fr
    Fetched_by:           amf2288
    Fetched_date:         2025/03/11
    Fetched_constraints:  [x=0.00/5.00; y=40.00/45.00; z=0.0/2002.0]
    Fetched_uri:          
    Valid uris:           ['https://data-argo.ifremer.fr/dac/coriolis/6903068...
    All uris:             ['https://data-argo.ifremer.fr/dac/coriolis/6903068...
    Processing_history:   [PRES,TEMP,PSAL] real-time and adjusted/delayed var...